In [ ]:
# Best Model Selection: FastText Word Embeddings Implementation

In [2]:
#Import Libraries
import os
import numpy as np
import tqdm
from tqdm import tqdm
import pandas as pd
#Text Pre-processing Using SpaCy and gensim
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_non_alphanum
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!python -m spacy download en_core_web_md
import spacy
from spacy.lang.en import English
from spacy import displacy
nlp = spacy.load('en_core_web_md')

2022-02-24 08:18:11.776616: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-02-24 08:18:11.776723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
You should consider upgrading via the 'C:\Users\ACER\anaconda3\python.exe -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
#Get the Data (Test on 3 chapters first)
data = pd.read_csv(r"C:\Users\Acer\anaconda3\activity\Sahih Bukhari Dataset (20 Chapters).csv") 
data

,Source,Volume_No,Chapter_No,Hadith_No,Chapter,Text
0,Sahih Bukhari,1,1,1,Revelation,Narrated by 'Umar bin Al-Khattab: I heard All...
1,Sahih Bukhari,1,1,2,Revelation,Narrated by 'Aisha: (the mother of the faithfu...
2,Sahih Bukhari,1,1,3,Revelation,Narrated by 'Aisha: (the mother of the faithfu...
3,Sahih Bukhari,1,1,4,Revelation,Narrated by Said bin Jubair: Ibn 'Abbas in the...
4,Sahih Bukhari,1,1,5,Revelation,Narrated by Ibn 'Abbas: Allah's Apostle was th...
...,...,...,...,...,...,...
1050,Sahih Bukhari,2,20,216,Shortening Prayers (At-Taqseer),Narrated by 'Imran bin Husain (who had piles):...
1051,Sahih Bukhari,2,20,217,Shortening Prayers (At-Taqseer),Narrated by 'Abdullah bin Buraida: 'Imran bin ...
1052,Sahih Bukhari,2,20,218,Shortening Prayers (At-Taqseer),"Narrated by 'Imran bin Husain had piles, so I ..."
1053,Sahih Bukhari,2,20,219,Shortening Prayers (At-Taqseer),Narrated by 'Aisha (the mother of the faithful...


In [4]:
# Use spaCy to split each of the documents into a list of words (tokenization).
# Clean the data by removing stop words, punctuation and converting to lowercase using the Gensim library

#creates a list of documents with a list of words inside:
text = []
for i in data.Text.values:
  doc = nlp(remove_stopwords(strip_punctuation(strip_non_alphanum(str(i).lower()))))
  tokens = [token.text for token in doc]
  text.append(tokens)

In [5]:
# Create the relevant phrases from the list of sentences:
phrases = Phrases(text, threshold = 10, min_count=5)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)
# Applying the Phraser to transform our sentences is simply
tokens = list(bigram[text])

In [6]:
#Train the fastText Model

model = FastText(tokens, vector_size=100, window=3, min_count=1, epochs=10, sorted_vocab=1)

In [7]:
#Create TF-IDF Scores for each Word

#Needs a list of lists for words and docs along with a fasttext 'model'
text = []
for i in tqdm(tokens):
  string = ' '.join(i)
  text.append(string)
tf_idf_vect = TfidfVectorizer(stop_words=None)
final_tf_idf = tf_idf_vect.fit_transform(text)
tfidf_feat = tf_idf_vect.get_feature_names()

100%|██████████| 1055/1055 [00:00<00:00, 211156.27it/s]
C:\Users\ACER\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
#Apply the Scores to the Vectors across each Document

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
errors=0
for sent in tqdm(tokens): # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf [row, tfidf_feat.index(word)]
            sent_vec += (vec * tfidf)
            weight_sum += tfidf
        except:
            errors =+1
            pass
    sent_vec /= weight_sum
    #print(np.isnan(np.sum(sent_vec)))

    tfidf_sent_vectors.append(sent_vec)
    row += 1
print('errors noted: '+str(errors))

100%|██████████| 1055/1055 [00:10<00:00, 100.64it/s]

errors noted: 1


In [9]:
# join these vectors back to the dataframe:
data['FT_tfidf'] = tfidf_sent_vectors

In [21]:
# Check word embedding for a perticular word
model.wv['medina']

array([-0.06987305,  0.4741976 , -0.6475395 ,  0.39263478,  0.56583434,
        0.17668872,  0.642225  ,  0.01475475,  0.34737426, -0.6311876 ,
       -0.2025631 ,  0.04484559, -0.8289995 ,  0.36812982,  0.09865559,
       -0.13964562,  0.5018593 , -0.29955   , -0.33617982, -0.16306382,
       -0.682048  ,  0.06868511, -0.3024476 , -0.05793094,  0.00243659,
       -0.6006213 , -0.50168467,  0.0079542 ,  0.4247161 , -0.04038078,
        0.1105122 ,  0.2602314 ,  0.91100353, -0.1385204 , -0.17607686,
        0.4633892 ,  0.6164109 ,  0.15726954, -0.57513267,  0.22892094,
       -0.12922685, -0.10558337,  0.4025497 , -0.54124427, -0.16490866,
       -0.01009859, -0.2361288 ,  0.23114832,  0.2411838 , -0.093402  ,
        0.24413407,  0.16332717,  0.5229539 ,  0.0597644 ,  0.20068686,
        0.45073047, -0.1624035 , -0.3003649 , -0.2641128 , -0.44358578,
        0.0734899 , -0.1792657 , -0.37934962,  0.5425251 , -0.05034773,
       -0.12800488,  0.09676318,  0.19764064,  0.1379782 ,  0.70

In [22]:
#Dimension must be 100 due to set the vector_size=100
model.wv['medina'].shape

(100,)

In [18]:
# Check the similarity of the model (wv=words vector)
# Try misspelling heaven as ‘heaveen’

similarities = model.wv.most_similar(restrict_vocab=10000, positive=['heaveen'])
similarities

[('heaven', 0.9999942779541016),
 ('heavens', 0.9999906420707703),
 ('heavily', 0.9999815225601196),
 ('graves', 0.9999745488166809),
 ('health', 0.9999741315841675),
 ('hen', 0.9999733567237854),
 ('beneath', 0.9999719858169556),
 ('behaves', 0.9999701976776123),
 ('lavatories', 0.9999698400497437),
 ('hearts', 0.9999693036079407)]

In [23]:
# Check top 10 similarity score between two word
sim_score = model.wv.similarity('medina', 'virtue')
sim_score

0.9997079

In [28]:
# Most opposite to a word
model.wv.most_similar(negative=["medina"])

[('24', -0.5633124709129333),
 ('488', -0.5641019344329834),
 ('c', -0.7189003825187683),
 ('657', -0.7396385669708252),
 ('74', -0.8287102580070496),
 ('narrated_abu', -0.8488999009132385),
 ('39', -0.8492275476455688),
 ('narrated_abdullah', -0.853453516960144),
 ('36', -0.8542405366897583),
 ('narrated_nafi', -0.8569827675819397)]

In [29]:
#Find Similar Documents (Query = good deeds)
from simple_colors import *

queryText = 'good deeds' 

query = [data.loc[data.Text.str.contains(queryText)].iloc[0]['FT_tfidf']]
query = np.array(list(query))
query = np.nan_to_num(query)

vectors = np.array(list(data.FT_tfidf.values))
vectors = np.nan_to_num(vectors)

cosine_similarities = pd.Series(cosine_similarity(query, vectors).flatten())

for i,j in cosine_similarities.nlargest(5).iteritems():
  print ("\n")
  print("Accuracy: " + str(j))
  #print("Index: "+str(i))
  print ("Book: " + data.Chapter.iloc[i])
  print ("Volume: " + str(data.Volume_No.iloc[i]))
  print ("Hadith No: " + str(data.Hadith_No.iloc[i]))
  print ("\n" + (green(data.Text.iloc[i],'bold')))
  print ("Source: " + data.Source.iloc[i])



Accuracy: 0.9999999999999998
Book: Belief
Volume: 1
Hadith No: 40

Narrated by Abu Huraira: Allah's Apostle said, "If any one of you improve (follows strictly) his Islamic religion then his good deeds will be rewarded ten times to seven hundred times for each good deed and a bad deed will be recorded as it is."
Source: Sahih Bukhari


Accuracy: 0.9999898313058528
Book: Ablution (Wudu')
Volume: 1
Hadith No: 184

Narrated by Asma' bint Abu Bakr: I came to 'Aisha the wife of the Prophet during the solar eclipse. The people were standing and offering the prayer and she was also praying. I asked her, "What is wrong with the people?" She beckoned with her hand towards the sky and said, "Subhan Allah." I asked her, "Is there a sign?" She pointed out, "Yes." So I, too, stood for the prayer till I fell unconscious and later on I poured water on my head. After the prayer, Allah's Apostle praised and glorified Allah and said, "Just now I have seen something which I never saw before at this plac